# Test Distributional Shift
This notebook tests distributional shift for the dataset SQuAD.

## Imports

In [1]:
%%capture
!pip install transformers

In [2]:
%%capture
!pip install datasets

In [3]:
%%capture
!pip install nltk

In [4]:
%%capture
import nltk
nltk.download('punkt')

In [5]:
from transformers import BertTokenizer, BertModel, AlbertTokenizer, AlbertModel, DistilBertTokenizer, DistilBertModel, RobertaTokenizer, RobertaModel, ElectraTokenizer, ElectraModel
import torch
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from math import ceil

from nltk.tokenize import sent_tokenize, word_tokenize
import random
import numpy as np

## Loading and Preparing Texts

In [6]:
def remove_sentence(t, k=1):
    sentences = random.sample(sent_tokenize(t), k)
    text = t
    for s in sentences:
        text = text.replace(s, '')
        assert len(t) != len(text)
    return text, len(word_tokenize(s))

def remove_word(t, k=1):
    text = t
    for _ in range(k):
        words = word_tokenize(text)
        if not len([i for i,w in enumerate(words) if w.isalnum()]):
            break
        word_idx = random.choice([i for i,w in enumerate(words) if w.isalnum()])
        del words[word_idx]
        text = ''.join([(' ' if w.isalnum() else '')+w for w in words]).strip(' ')
    return text

In [7]:
dataset = load_dataset('squad', split='train')
print('Average number of sentences = {}'.format(np.mean([len(sent_tokenize(sample['context'])) for sample in dataset ])))

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41. Subsequent calls will reuse this data.
Average number of sentences = 5.1039509583442735


In [8]:
N_WORD = 5
texts = []
dataset = load_dataset('squad', split='train')
for d in tqdm(dataset):
    if d['context'] not in texts:
        texts.append(d['context'])
random.shuffle(texts)        
n_samples = len(texts)
original = texts#random.sample(texts, n_samples)#texts[:n_samples]
word_removed = []
sentence_removed = []
for t in tqdm(original):
    sr, nw = remove_sentence(t)
    wr = remove_word(t, k=N_WORD)
    word_removed.append(wr)
    sentence_removed.append(sr)

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41)
100%|██████████| 18891/18891 [01:35<00:00, 197.37it/s]


In [9]:
text_data = original+word_removed+sentence_removed
classes = [
    (n_samples, 'Original', 'tab:blue'),
    (n_samples, 'Word', 'tab:green'),
    (n_samples, 'Sentence', 'tab:red')
]

## Compare Distributional Shift

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [11]:
#def get_accuracy_rf(x_original, x_new):
#  X = torch.cat([x_original, x_new], dim=0).cpu().numpy()
#  y = np.array([0]*n_samples+[1]*n_samples)
#  X_train, X_test, y_train, y_test = train_test_split(X, y)
#  clf = RandomForestClassifier()
#  clf.fit(X_train, y_train)
#  print('Train : {}'.format(clf.score(X_train, y_train)))
#  print('Test : {}'.format(clf.score(X_test, y_test)))
def get_accuracy_rf(x_original, x_new):
  X = torch.cat([x_original, x_new], dim=0).cpu().numpy()
  y = np.array([0]*n_samples+[1]*n_samples)
  X_train, X_test, y_train, y_test = train_test_split(X, y)
  oob_score = 0
  best_clf = None
  for depth in [2, 5, 7, 10, 15, 20]:
    clf = RandomForestClassifier(max_depth=depth, oob_score=True)
    clf.fit(X_train, y_train)
    print('Depth {}, OOB score {}'.format(depth, clf.oob_score_))
    if clf.oob_score_>oob_score:
      oob_score = clf.oob_score_
      best_clf = clf
  print('--> Depth={}'.format(best_clf.max_depth))
  print('-'*20)
  train_acc = best_clf.score(X_train, y_train)
  test_acc = best_clf.score(X_test, y_test)
  print(X_train.shape)
  print('Train : {}({})'.format(train_acc, train_acc*(1-train_acc)/X_train.shape[0]))
  print('Test : {}({})'.format(test_acc, test_acc*(1-test_acc)/X_test.shape[0]))
  print('-'*20)

In [12]:
def process_texts(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return model(**(inputs.to(device)))[0][:, 0, :].cpu().detach()

def batch(l, size):
    for i in range(0, len(l), size):
        yield l[i:i+size]

In [13]:
def models_generator():
  model_classes = [
                   BertModel,
                   RobertaModel
  ]
  tokenizer_classes = [
                       BertTokenizer,
                       RobertaTokenizer
  ]
  model_names = [
                 'bert-base-uncased',
                 'roberta-base'
  ]
  for model_name, tokenizer_class, model_class in zip(model_names, tokenizer_classes, model_classes):
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model = model_class.from_pretrained(model_name).to(device)
    yield model, tokenizer, model_name.split('-')[0].upper()

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
for model, tokenizer, name in models_generator():
  print('Model: {}'.format(name))
  results = []
  for b in batch(text_data, 5):
      results.append(process_texts(b, model, tokenizer))
  embeddings = torch.cat(results, 0)
  e_original = embeddings[:n_samples, :]
  e_word = embeddings[n_samples:2*n_samples, :]
  e_sentence = embeddings[2*n_samples:, :]
  print('Sentence vs Original')
  get_accuracy_rf(e_original, e_sentence)
  print('Word vs Original')
  get_accuracy_rf(e_original, e_word)
  print('\n\n')


Model: BERT
Sentence vs Original
Depth 2, OOB score 0.5679700734048561
Depth 5, OOB score 0.5788043478260869
Depth 7, OOB score 0.5719226425748165
Depth 10, OOB score 0.5377611518915867
Depth 15, OOB score 0.46904997176736307
Depth 20, OOB score 0.4331239412761152
--> Depth=5
--------------------
(28336, 768)
Train : 0.6102837380011293(8.39347463059353e-06)
Test : 0.5729409273766674(2.5902987625813227e-05)
--------------------
Word vs Original
Depth 2, OOB score 0.7744918125352908
Depth 5, OOB score 0.7946075663466968
Depth 7, OOB score 0.8005011293054771
Depth 10, OOB score 0.7976778656126482
Depth 15, OOB score 0.7863495200451722
Depth 20, OOB score 0.7795031055900621
--> Depth=7
--------------------
(28336, 768)
Train : 0.8432382834556748(4.665001438819942e-06)
Test : 0.8121956383654457(1.6147986807706102e-05)
--------------------






Model: ROBERTA
Sentence vs Original
Depth 2, OOB score 0.7347190852625636
Depth 5, OOB score 0.7528938452851497
Depth 7, OOB score 0.7577639751552795
Depth 10, OOB score 0.7472826086956522
Depth 15, OOB score 0.7136857707509882
Depth 20, OOB score 0.7012634105025409
--> Depth=7
--------------------
(28336, 768)
Train : 0.8169819311123659(5.276766493092719e-06)
Test : 0.7607452890112216(1.9268673963418856e-05)
--------------------
Word vs Original
Depth 2, OOB score 0.8149703557312253
Depth 5, OOB score 0.8419325239977414
Depth 7, OOB score 0.8567193675889329
Depth 10, OOB score 0.8610601355166573
Depth 15, OOB score 0.8576722190852626
Depth 20, OOB score 0.8547783738001129
--> Depth=10
--------------------
(28336, 768)
Train : 0.932806324110672(2.2119807245132678e-06)
Test : 0.8582468769849672(1.2879438400435932e-05)
--------------------





In [16]:
model = DistilBertModel.from_pretrained('distilbert-base-uncased').to(device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [17]:
print('Model: DistilBert')
results = []
for b in batch(text_data, 2):
    results.append(process_texts(b, model, tokenizer))
embeddings = torch.cat(results, 0)
e_original = embeddings[:n_samples, :]
e_word = embeddings[n_samples:2*n_samples, :]
e_sentence = embeddings[2*n_samples:, :]
print('Sentence vs Original')
get_accuracy_rf(e_original, e_sentence)
print('Word vs Original')
get_accuracy_rf(e_original, e_word)
print('\n\n')

Model: DistilBert
Sentence vs Original
Depth 2, OOB score 0.5552300959909655
Depth 5, OOB score 0.5523362507058159
Depth 7, OOB score 0.532608695652174
Depth 10, OOB score 0.47568464144551104
Depth 15, OOB score 0.37567052512704685
Depth 20, OOB score 0.3212521174477696
--> Depth=2
--------------------
(28336, 768)
Train : 0.5756634669678148(8.620660635460559e-06)
Test : 0.5599195426635613(2.6086136820557875e-05)
--------------------
Word vs Original
Depth 2, OOB score 0.754305477131564
Depth 5, OOB score 0.7723743647656691
Depth 7, OOB score 0.7780561829474872
Depth 10, OOB score 0.773538961038961
Depth 15, OOB score 0.7441064370412197
Depth 20, OOB score 0.737789384528515
--> Depth=7
--------------------
(28336, 768)
Train : 0.8260516657255788(5.070945485515791e-06)
Test : 0.7884818970993013(1.7655959670335396e-05)
--------------------





In [18]:
model = ElectraModel.from_pretrained('google/electra-small-discriminator').to(device)
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

In [19]:
print('Model: Electra')
results = []
for b in batch(text_data, 2):
    results.append(process_texts(b, model, tokenizer))
embeddings = torch.cat(results, 0)
e_original = embeddings[:n_samples, :]
e_word = embeddings[n_samples:2*n_samples, :]
e_sentence = embeddings[2*n_samples:, :]
print('Sentence vs Original')
get_accuracy_rf(e_original, e_sentence)
print('Word vs Original')
get_accuracy_rf(e_original, e_word)
print('\n\n')

Model: Electra
Sentence vs Original
Depth 2, OOB score 0.5875917560700169
Depth 5, OOB score 0.5955321852060983
Depth 7, OOB score 0.5924265951439864
Depth 10, OOB score 0.5735107284020328
Depth 15, OOB score 0.5267151326933935
Depth 20, OOB score 0.49947063805759456
--> Depth=5
--------------------
(28336, 256)
Train : 0.6207298136645962(8.308311409250136e-06)
Test : 0.5952784247300444(2.5505189686741612e-05)
--------------------
Word vs Original
Depth 2, OOB score 0.8794466403162056
Depth 5, OOB score 0.9001623376623377
Depth 7, OOB score 0.9108201581027668
Depth 10, OOB score 0.9162196499153021
Depth 15, OOB score 0.9191487859966121
Depth 20, OOB score 0.9184782608695652
--> Depth=15
--------------------
(28336, 256)
Train : 0.9874364765669114(4.3780637041342724e-07)
Test : 0.9237772602159644(7.454248753318949e-06)
--------------------



